# Remote connection to jupyter notebooks

<span style="font-size:small;color:gray;">
<a href="https://nbviewer.jupyter.org/github/birnstiel/jupyter-intro/blob/master/jupyter-intro.ipynb">go here for Jupyter Notebook intro</a>
</span>

# Why

Say you have lots of data on a remote machine:


- working locally may be hard (little disk-space) ...
- or impossible (little memory)
- you have to manage both machines

Working remotely is the solution, but comes with its own challenges, mainly the **connection**:

> when connecting through SSH, connection might be lost $\rightarrow$ need to start again 

Better solution: have the kernel / namespace / ... running on the remote machine but being able to reconnect to it.
    
$\rightarrow$ this can be done for the terminal (see later), but is conveniently done with a *notebook server*

## On the host

### Set a password

Most importantly, we want to set a password. You can do that by calling

    > jupyter notebook password
    
enter a password twice. The hash of the password is written into a configuration file as well:

    [NotebookPasswordApp] Wrote hashed password to /Users/USERNAME/.jupyter/jupyter_notebook_config.json

Not setting a password will cause jupyter to generate a token each time that you need to pass when logging in.

### Use https

To make the communication more secure, we want to communicate with the server using https. The easiest way is to use a self-made certificate. The downside is that browsers will warn you about this and that some operating systems or applications might not work with it (e.g. iOS).

To generate those use this command:
    
    > openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout mykey.key -out mycert.pem
  

This will ask you some questions. What you put in doesn't really matter.
    

    Generating a RSA private key
    ..............................................................................................................+++++
    ...............................................................+++++
    writing new private key to 'mykey.key'
    -----
    You are about to be asked to enter information that will be incorporated
    into your certificate request.
    What you are about to enter is what is called a Distinguished Name or a DN.
    There are quite a few fields but you can leave some blank
    For some fields there will be a default value,
    If you enter '.', the field will be left blank.
    -----
    Country Name (2 letter code) [AU]:<span style="color:red">DE</span>
    State or Province Name (full name) [Some-State]:<span style="color:red">BY</span>
    Locality Name (eg, city) []:<span style="color:red">Munich</span>
    Organization Name (eg, company) [Internet Widgits Pty Ltd]:<span style="color:red">LMU</span>
    Organizational Unit Name (eg, section) []:<span style="color:red">USM</span>
    Common Name (e.g. server FQDN or YOUR name) []:<span style="color:red">Til Birnstiel</span>
    Email Address []:til.birnstiel@lmu.de


The created files `mycert.pem` and `mykey.key` are in the current directory. You can put them somewhere useful, but then use absolute pathes to point to them.

### Start the server

The server should be running for a while, therefore I recommend to use `screen` or `tmux` to run it in the background and to be able to log out without terminating the server.

#### `screen`
    screen -S notebook
    [start the notebook]
    [press CTRL+A D to detach]

#### `tmux`

    tmux
    [start the notebook]
    [press CTRL-B D to detach]

We want to start the server using a couple of settings. We can do this in the command line:
    
    
    jupyter notebook --ip='0.0.0.0' --certfile=mycert.pem --keyfile=mykey.key --no-browser --port=8888
    

Here, we:

- start a notebook server
- tell the server to listen to local connections. If your machine is visible to the internet, use it's global IP instead.
- encrypt the traffic using our certificate/key
- not start the browser (since we want to connect from somewhere else)
- we use port 8888. That's the default for jupiter, so leaving this out will not change something. But if you use a machine with more users trying to set up servers, you need to find one that is not in use. Leaving out the `--port` will cause jupyter to select the next available port (8889, 8890), but you need to then note that port.

I recommend to start the server in the home directory so that you have access to all your files. If you store your certificate/key somewhere else you need to give the appropriate path like `--certfile=/path/to/mycert.pem`.

### Using a config file

Instead of using this long command, you can also store your default settings in the configuration file. Here's the contents of `~/.jupyter/jupyter_notebook_config.py`:

	c.NotebookApp.certfile = u'/home/moon/USERNAME/jupyter_certs/jupyter/certs/ssl.cert.pem'
	c.NotebookApp.keyfile  = u'/home/moon/USERNAME/jupyter_certs/jupyter/private/ssl.key.pem'
	c.NotebookApp.ip = '129.187.204.168'
	c.NotebookApp.password = u'sha1:XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
	c.NotebookApp.open_browser = False
	c.NotebookApp.port = 8888

Here, I set the IP to be the externally visible IP of the server. This way, just invoking `jupyter notebook` should be enough.

A template configuration file can be created by calling

    jupyter notebook --generate-config

    Writing default config to: /Users/USERNAME/.jupyter/jupyter_notebook_config.py

# Connect

That was all we had to do on the remote host. We can now log out and try to connect. Again there are different ways:

- if your machine is globally visible, you can just go to a browser and enter

        https://remote-server.de:8888
    
    where 8888 would be replaced with the port you are using.

    
- if your machine is not directly accessible, you can use an SSH tunnel to connect.

        ssh -N -L 9999:localhost:8888 USERNAME@remote-server.de
        
    This command will not return the prompt or produce output. Again, you could keep it running in a `screen` session. This will open a tunnel from your local port 9999 (or whatever you want to use) to the port 8888 on the remote machine (where your server is running). Now you should be able to open this address in your browser:
    
        https://localhost:9999
      

  
**In both cases:** make sure you use `https` in the address if you use the certificats. `http` will not work and vice versa.

# Extras

### going through a Login Machine

If you cannot login directly to your machine, you need to tunnel to it:
    
    ssh -L 9999:TARGETHOSTNAME:8888 -N USERNAME@LOGINMACHINE.de

So this means:

    ____________________________       __________________      __________________________
    | your computer, Port 9999 |  ---> |  LOGINMACHINE  | ---> | TARGET HOST, Port 8888 |
    ____________________________       __________________      __________________________
                                        visible to world        hidden behind LOGINMACHINE

### Remote terminal only

If you prefer working just in the IPython console, you can do that also remotely

1. start a kernel

        ipython kernel
        
	note the `kernel-XXXX.json` name that is printed to screen. It contains the connection info.

2. on local machine, download that file:

        scp USER@REMOTEMACHINE.de:~/.ipython/profile_default/security/kernel-*.json .
		
	if it's not there, it's in `jupyter --runtime-dir`
		
	and then connect to the right kernel file if there are several
	
		jupyter qtconsole --existing /Users/USERNAME/kernel-XXXXX.json --ssh USERNAME@REMOTEMACHINE.de

### Jupyter Lab

If you haven't tried it yet, replace

    jupyter notebook
    
with

    jupyter lab

Jupyter Lab is a single browser-window interface to a notebook server. It's much more user-friendly and a much cleaner interface. <span style="color:rgb(50,50,50)">Just interactive figures do not work so well yet.</span>

It's part of anaconda already, or you quickly install it with `pip install jupyterlab`.

<img src="https://jupyterlab.readthedocs.io/en/stable/_images/jupyterlab.png" width=50%/>

### Jupyter Hub

For multi-user environments, I recommend having a look at [Jupyter Hub](https://jupyter.org/hub). This is a structure on top of the notebook server where (for example) every user of the machine can start their own server. One can also allow other users to register/log in, for example for classes or tutorials. Setup can be challenging. Simplest setup is probably [this one](http://tljh.jupyter.org/en/latest/).

### iOS

There is an app for jupyter. It works great if you have a keyboard, apart from the missing <kbd>ESC</kbd> Key. To get it running you need to either use SSH forwarding, or follow these instructions to install a self-signed certificate on your iOS device:

    https://juno.sh/ssl-self-signed-cert/

### Browser Tab

I found it helpful to rename the browser tab if I am connected to several notebook servers in different tabs. You can do that by executing this in one of the notebook cells:
```javascript
%%javascript  
document.title='GustlLab'
```

In [ ]:
!jupyter nbconvert --to slides --post serve remote_jupyter.ipynb